In [ ]:
import numpy as np
import scipy.stats as st
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, AveragePooling2D, Flatten, Dense, Lambda
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.losses import mean_squared_error, mean_absolute_error, categorical_crossentropy
from keras.preprocessing.image import Iterator
import matplotlib.pyplot as plt
import cv2
import os
import json
import pickle
%matplotlib inline

In [ ]:
from keras.utils.np_utils import to_categorical

In [ ]:
from keras.layers import Reshape, BatchNormalization

In [ ]:
import tensorflow as tf
sess = tf.Session()
K.set_session(sess)

In [ ]:
def sum_squared_error(y_true, y_pred):
    return K.sum(K.sum(K.sum(K.square(y_pred - y_true), axis=-1), axis=-1), axis=-1)

In [ ]:
def count_rmse(y_true, y_pred):
    """ Compute the count rmse from the gt and predicted density map.
    :param y_true: Groundtruth density map, 4D tensor of shape (n_samples, height, width, types of sealion)
    :param y_pred: Predicted density map, 4D tensor of shape (n_samples, height, width, types of sealion)
    :return A Tensor of shape () (single value), with the RMSE(y_true, y_pred), averaged over the columns.
    """
    # shape is (n_samples, height, width, types of sealion)
    # we want to reduce it to (n_samples, types of sealion)
    # We don't round the counts, as it will be called on patches, with probably non-integer sum over the density map.
    counts_true = K.sum(K.sum(y_true, axis=1), axis=1)
    counts_pred = K.sum(K.sum(y_pred, axis=1), axis=1)
    rmse_per_column = K.sqrt(K.mean(K.square(counts_pred - counts_true), axis=0))
    rmse = K.mean(rmse_per_column)
    return rmse

In [ ]:
def count_mse(y_true, y_pred):
    """ Compute the count mse from the gt and predicted density map.
    :param y_true: Groundtruth density map, 4D tensor of shape (n_samples, height, width, types of sealion)
    :param y_pred: Predicted density map, 4D tensor of shape (n_samples, height, width, types of sealion)
    :return A Tensor of shape (n_samples,), with the MSE(count_true, count_pred) for each sample.
    """
    # shape is (n_samples, height, width, types of sealion)
    # we want to reduce it to (n_samples, types of sealion)
    # We don't round the counts, as it will be called on patches, with probably non-integer sum over the density map.
    counts_true = K.sum(K.sum(y_true, axis=1), axis=1)
    counts_pred = K.sum(K.sum(y_pred, axis=1), axis=1)
    rmse_per_sample = K.square(counts_pred - counts_true)
    return rmse_per_sample

In [ ]:
def np_count_mse(y_true, y_pred):
    """ Compute the count mse from the gt and predicted density map.
    :param y_true: Groundtruth density map, 4D tensor of shape (n_samples, height, width, types of sealion)
    :param y_pred: Predicted density map, 4D tensor of shape (n_samples, height, width, types of sealion)
    :return A Tensor of shape (n_samples,), with the MSE(count_true, count_pred) for each sample.
    """
    # shape is (n_samples, height, width, types of sealion)
    # we want to reduce it to (n_samples, types of sealion)
    # We don't round the counts, as it will be called on patches, with probably non-integer sum over the density map.
    counts_true = np.sum(np.sum(y_true, axis=1), axis=1)
    counts_pred = np.sum(np.sum(y_pred, axis=1), axis=1)
    rmse_per_sample = np.square(counts_pred - counts_true)
    n, h, w, c = y_true.shape
    return rmse_per_sample/(w * h)

In [ ]:
def get_unet(n_channels, n_classes, lr=0.001):
    # Fully convolutional, we don't specify the image size
    inputs = Input((None, None, n_channels))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    #conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    #conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    #conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    #conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    #conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)

    conv10 = Conv2D(n_classes, (1, 1), activation=None)(conv9)  # outputs N density map, one for each type of sealion
    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=lr), loss=mean_squared_error)

    return model

In [ ]:
def get_cnet(img_rows, img_cols, n_channels, n_classes, lr=0.001):
    inputs = Input((img_rows, img_cols, n_channels))
    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    #conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    #conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    #conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    #conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    #conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)

    avg = AveragePooling2D((7, 7))(conv5)   #(h/(16 * 7), w/(16 * 7), 512)
    avg = Flatten()(avg)
    avg = BatchNormalization()(avg)
    
    hidden = Dense(128, activation='relu')(avg)
    hidden = BatchNormalization()(hidden)
    counts = Dense(n_classes, activation='softmax')(hidden)
    
    model = Model(inputs=[inputs], outputs=[counts])

    model.compile(optimizer=Adam(lr=lr), loss=categorical_crossentropy)

    return model

In [ ]:
def get_block_loc(shape, x, y, target_size=(224, 224), n_blocks=(4,4), overlap=(448,448)):
    h, w = shape
    w_block = (w + (n_blocks[1] - 1) * overlap[1]) // n_blocks[1]
    h_block = (h + (n_blocks[0] - 1) * overlap[0]) // n_blocks[0]
    for by in range(n_blocks[0]):
        y_start = by * (h_block - overlap[0])
        y_end = y_start + h_block + 1
        for bx in range(n_blocks[1]):
            x_start = bx * (w_block - overlap[1])
            x_end = x_start + w_block + 1
            
            if x_start <= x < x_end and y_start <= y < y_end and\
            x_start <= x + target_size[1] - 1 < x_end and y_start <= y + target_size[0] - 1 < y_end:
                return bx + by * n_blocks[0], x - x_start, y - y_start
    raise Exception("Can't find block...??")

In [ ]:
class PatchIterator(Iterator):
    """Iterator yielding training samples
    :param root_dir: Directory containing training images, density map and sampling map.
    :param image_ids: Set of image ids to use to sample patches.
    :param x_mean: Mean per channel for input normalization
    :param x_std: Standard deviation per channel for input normalization
    :param y_scale: Scale to apply on the output density map
    :param output_counts: Indicate if the iterator should return only counts, otherwise density maps.
    :param n_samples_per_image: Number of patches to sample on each image.
    :param target_size: Size of the patches sampled.
    :param batch_size: Number of patches sampled per batch
    :param shuffle: Boolean, whether to shuffle the data between epochs.
    :param seed: Random seed for data shuffling.
    :return batch_x, batch_x. 
        batch_x is a (batch_size, target_size[0], target_size[1], 3) array
        batch_x is a (batch_size, target_size[0], target_size[1], 1) array if output_counts is False
        otherwise, it is a (batch_size, 5) array.
    """

    def __init__(self, root_dir, image_ids,
                 x_mean, x_std, y_scale,
                 output_counts=False,
                 count_max=64,
                 n_samples_per_image=160,
                 target_size=(224, 224),
                 batch_size=8, shuffle=True, seed=42):
        
        self.image_ids = image_ids
        self.max_count = count_max
        self.x_mean = x_mean
        self.x_std = x_std
        self.y_scale = y_scale
        self.output_counts = output_counts
        self.root_dir = root_dir
        self.n_samples_per_image = n_samples_per_image
        self.target_size = target_size
        self.n_indices = len(self.image_ids) * self.n_samples_per_image
                 
        super(PatchIterator, self).__init__(self.n_indices, batch_size, shuffle, seed)
        
    def __normalize_input__(self, batch_x):
        return (batch_x/255 - self.x_mean)/self.x_std
    
    def __normalize_output__(self, batch_y):
        ma = 0.006
        mi = 0
        a = 1/(ma - mi)
        b = -a * mi
        mean = 4.1060985197746936e-06
        std = 1.12463456142464e-05
        scale_mean = a * mean + b
        scale_std = a * std
        log_mean = np.log(scale_mean)
        log_std = np.log(std)
        scaled_01 =  (a * batch_y + b)
        log = np.log(scaled_01 + 1e-9)
        return ((log - log_mean)/log_std) * self.y_scale

    def __sample__(self, shape, dots, image_id):
        pbackground = 0.2
        threshold_masked = 0.3 # if more than 30% of the patch is masked, reject it
        
        adots = []
        for _, ds in dots.items():
            for x, y in ds:
                if 0 <= x < shape[1] and 0 <= y < shape[0]:
                    adots.append((x, y))
                    
        bg_or_dots = np.random.choice(2, size=(1, ), p=[pbackground, 1-pbackground])
        
        while 1:
            if bg_or_dots[0] == 0 or len(adots) == 0:
                x, y = self.sample_bg(shape, image_id)
            else:
                x, y = self.sample_dot(shape, adots, image_id)

            try:
                bid, x, y = get_block_loc(shape, x, y)
            except:
                continue
            uid = "{iid}_{bid}".format(iid=image_id, bid=bid)
            img = cv2.imread(os.path.join(self.root_dir, "TrainBlock", uid + ".jpg"))/255
            img_patch = img[y:y+self.target_size[0], x:x+self.target_size[1],:]
            masked_pixels = np.count_nonzero(img_patch == 0)
            total_pixels = img_patch.shape[0] * img_patch.shape[1]
            if img_patch.shape[0] != self.target_size[0] or img_patch.shape[1] != self.target_size[1]:
                continue
            if masked_pixels/total_pixels < threshold_masked:
                dmap = np.load(os.path.join(self.root_dir, "TrainDensity", uid + ".npz"))['dmap']
                return img_patch, dmap[y:y+self.target_size[0], x:x+self.target_size[1],:]
        
    def sample_bg(self, shape, image_id):
        x = np.random.randint(0, shape[1] - self.target_size[1], size=(1,))[0]
        y = np.random.randint(0, shape[0] - self.target_size[0], size=(1,))[0]
        
        return x, y
    
    def sample_dot(self, shape, adots, image_id):
        half_size = 36
        
        randi = np.random.choice(len(adots), size=(1,))[0]
        rand_dot = adots[randi]
        
        min_x = max(0, rand_dot[0] - self.target_size[1] + half_size)
        max_x = min(shape[1] - self.target_size[1], rand_dot[0] + self.target_size[1] - half_size)
        
        min_y = max(0, rand_dot[1] - self.target_size[0] + half_size)
        max_y = min(shape[0] - self.target_size[0], rand_dot[1] + self.target_size[0] - half_size)
        
        if min_x >= max_x:
            max_x, min_x = min_x, max_x
        if min_y >= max_y:
            max_y, min_y = min_y, max_y 
           
        x = np.random.randint(min_x, max_x, size=(1,))[0]
        y = np.random.randint(min_y, max_y, size=(1,))[0]
        
        return x, y
        
    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch.
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
                 
        batch_x = np.zeros((current_batch_size, self.target_size[0], self.target_size[1], 3), dtype=K.floatx())
        batch_y = np.zeros((current_batch_size, self.max_count)) if self.output_counts\
        else np.zeros((current_batch_size, self.target_size[0], self.target_size[1], 1), dtype=K.floatx())
        
        # For each index, we load the data and sample randomly n_successive_samples patches
        for i, j in enumerate(index_array):
            index = j // self.n_samples_per_image
            image_id = self.image_ids[index]
            with open(os.path.join(self.root_dir, "TrainDots", str(image_id) + ".pkl"), "rb") as pfile:
                dots = pickle.load(pfile)
            with open(os.path.join(self.root_dir, "TrainShape", str(image_id) + ".pkl"), "rb") as pfile:
                shape = pickle.load(pfile)
                
            x, y = self.__sample__(shape, dots, image_id)
            batch_x[i,:,:,:] = x
            if not self.output_counts: 
                batch_y[i,:,:] = np.sum(y, axis=-1, keepdims=True)
            else:
                counts = np.sum(y)
                batch_y[i, :] = to_categorical(np.round(counts).astype(np.int), num_classes=self.max_count)     

        return self.__normalize_input__(batch_x), self.__normalize_output__(batch_y)

In [ ]:
def show_density(im, density):
    max_density = np.max(density)
    if max_density > 0:
        normalized_density = density/max_density
    else:
        normalized_density = density
    im_fp32 = im.astype(np.float32)
    im_masked = im_fp32.copy()
    im_masked[:,:,0] = (im_fp32[:,:,0] * normalized_density)
    im_masked[:,:,1] = (im_fp32[:,:,1] * normalized_density)
    im_masked[:,:,2] = (im_fp32[:,:,2] * normalized_density)
    return im_masked.astype(np.uint8), (normalized_density * 255).astype(np.uint8)

In [ ]:
with open("../data/sealion/train.json", "r") as jfile:
    train_ids = json.load(jfile)
train_ids = [int(s[:-4]) for s in train_ids]

In [ ]:
with open("../data/sealion/dots_ok.json", "r") as jfile:
    dots_ok = json.load(jfile)
dots_ok = [int(s[:-4]) for s in dots_ok]

In [ ]:
train_ids = list(set(train_ids).intersection(set(dots_ok)))

In [ ]:
with open("../data/sealion/val.json", "r") as jfile:
    val_ids = json.load(jfile)
val_ids = [int(s[:-4]) for s in val_ids]
val_ids = list(set(val_ids).intersection(set(dots_ok)))

In [ ]:
mean_bgr = np.load("../data/sealion/mean_bgr.npy")
std_bgr = np.load("../data/sealion/std_bgr.npy")
trainPatchesGenerator = PatchIterator("/home/lowik/sealion/data/sealion/", train_ids, mean_bgr, std_bgr, 5, output_counts=False)
valPatchesGenerator = PatchIterator("/home/lowik/sealion/data/sealion/", val_ids, mean_bgr, std_bgr, 5, output_counts=False)

In [ ]:
for batch_x, batch_y in trainPatchesGenerator:
    break

In [ ]:
batch_y.shape

In [ ]:
for i, (x, y) in enumerate(zip(batch_x, batch_y)):
    im, mask = show_density(x, y[:,:,0])
    cv2.imwrite("../data/dmap" + str(i) + ".png", mask)
    #cv2.imwrite("../data/dmapx" + str(i) + ".png", x)

In [ ]:
unet = get_unet(3, 1, 0.001)
h = unet.fit_generator(trainPatchesGenerator, 100, epochs=2, verbose=1, callbacks=None, validation_data=valPatchesGenerator, validation_steps=20, class_weight=None, max_q_size=10, workers=2, pickle_safe=False, initial_epoch=0)

In [ ]:
for batch_x, batch_y in valPatchesGenerator:
    break

In [ ]:
batch_ypred = unet.predict(batch_x)

In [ ]:
batch_ypred.shape

In [ ]:
with sess.as_default():
    print(np_count_mse(batch_y, batch_ypred))

In [ ]:
with sess.as_default():
    print(np.mean(mean_squared_error(batch_y, batch_ypred).eval()))

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(batch_y[1,:,:,0])

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(batch_ypred[7,:,:,0])

In [ ]:
np.sum(batch_y)

In [ ]:
with sess.as_default():
    print(np.mean(mean_squared_error(batch_y, batch_ypred).eval()))